In [1]:
import numpy as np
import pandas as pd
import cv2
from scipy import interpolate
from scipy import ndimage
import scipy
from matplotlib import pyplot as plt
%matplotlib inline

<div class="alert alert-block alert-info">
<h3 class="alert-heading">Generate pairs of images and save them</h3>
</div>

In [2]:
# Uses the PyDIC lib (https://gitlab.com/damien.andre/pydic)
import imp
pydic = imp.load_source('pydic', '../pydic/pydic.py')

In [3]:
def CreateImage(rescaled_imgsize, scaling_factor):
    #Create background
    big_imgsize = int(rescaled_imgsize / scaling_factor)
    full_imgsize = big_imgsize
    img = np.full((full_imgsize, full_imgsize),
        np.random.randint(230,255),dtype = np.uint8)

    nb_speckles = np.random.randint(1e3,2e3)

    i=0
    while(i<nb_speckles):
        #random ellipse parameters
        center_val = (np.random.randint(0,full_imgsize),np.random.randint(0,full_imgsize))
        axis_len = (np.random.randint(8,18),np.random.randint(8,18))
        color_val = np.random.randint(0,90)
        
        #draw the ellipse
        cv2.ellipse(img,center_val,axis_len,0,0,360,color_val,-1)
        i += 1

    #resize image
    img = cv2.resize(img,(0,0),fx=scaling_factor, fy=scaling_factor,interpolation = 3) 
    return img

In [4]:
def deform_matrix(seed, shape, sigmas, smoothing_sigmas, base_max=10.0):
    x_location = np.random.randint(shape[0])
    y_location = np.random.randint(shape[1])
    x_displacement = np.random.uniform(-sigmas[0], sigmas[0])
    y_displacement = np.random.uniform(-sigmas[1], sigmas[1])
    dummy_xs = np.zeros(shape)
    dummy_ys = np.zeros(shape)
    dummy_xs[x_location, y_location] = x_displacement
    dummy_ys[x_location, y_location] = y_displacement
    smooth_xs = scipy.ndimage.gaussian_filter(dummy_xs, smoothing_sigmas[0])
    smooth_ys = scipy.ndimage.gaussian_filter(dummy_ys, smoothing_sigmas[1])
    smooth_xs = smooth_xs * (base_max / np.abs(smooth_xs).max())
    smooth_ys = smooth_ys * (base_max / np.abs(smooth_ys).max())
    return (smooth_xs, smooth_ys)

In [5]:
def shift_elements(original, deformations):
    x_def, y_def = deformations
    def shift_func(image):
        return (image[0] - x_def[image], image[1] - y_def[image])
    return ndimage.geometric_transform(original, shift_func)

In [6]:
def generate_pair(seed, displacement_sigmas, smoothing_sigmas, imsize=64, scaling_factor=0.1):
    orig = CreateImage(imsize, scaling_factor)
    deformations = deform_matrix(seed, orig.shape, displacement_sigmas, smoothing_sigmas)
    shifted = shift_elements(orig, deformations)
    return ((orig, shifted), deformations)

In [7]:
seeds = [23,9876,567,3,51,4509812]
for i in range(5):
    data = generate_pair(seed=seeds[i], displacement_sigmas=(20.0, 20.0), smoothing_sigmas=(7, 7))
    oriim = data[0][0]
    defim = data[0][1]
    cv2.imwrite( "./data/"+str(i)+"/00.tiff", oriim );
    cv2.imwrite( "./data/"+str(i)+"/01.tiff", defim );

<div class="alert alert-block alert-info">
<h3 class="alert-heading">Run DIC on images</h3>
</div>

In [12]:
correl_wind_size = (8,8)
correl_grid_size = (2,2)
for i in range(5):
    pydic = imp.load_source('pydic', '../pydic/pydic.py')
    pydic.init(image_pattern="./data/"+str(i)+"/*.tiff",
               win_size_px=correl_wind_size,
               grid_size_px=correl_grid_size,
               result_file="./data/"+str(i)+"/result.dic",
               area_of_intersest=[[0,0],[64,64]])
    pydic.read_dic_file("./data/"+str(i)+"/result.dic", interpolation='spline')
    #Get displacements
    last_grid = pydic.grid_list[-1]
    #write displacements to file
    df_dic = pd.DataFrame({'xpos':last_grid.grid_x.flatten(),
             'ypos':last_grid.grid_y.flatten(),
             'ux':last_grid.disp_x.flatten(),
             'uy':last_grid.disp_y.flatten()})
    df_dic.to_csv("./data/DIC"+str(i)+".csv",sep=';', index=False)
    

reading image 1 / 2 : "./data/0\01.tiff"

compute displacement and strain field of ./data/0\00.tiff ...
interpolate displacement with spline method
saving ./data/0/pydic/marker/00_marker.png file...
saving ./data/0/pydic/disp/00_disp.png file...
saving ./data/0/pydic/grid/00_grid.png file...
saving ./data/0/pydic/result/00_result.csv file...

compute displacement and strain field of ./data/0\01.tiff ...
interpolate displacement with spline method
saving ./data/0/pydic/marker/01_marker.png file...
saving ./data/0/pydic/disp/01_disp.png file...
saving ./data/0/pydic/grid/01_grid.png file...
saving ./data/0/pydic/result/01_result.csv file...
reading image 1 / 2 : "./data/1\01.tiff"

compute displacement and strain field of ./data/1\00.tiff ...
interpolate displacement with spline method
saving ./data/1/pydic/marker/00_marker.png file...
saving ./data/1/pydic/disp/00_disp.png file...
saving ./data/1/pydic/grid/00_grid.png file...
saving ./data/1/pydic/result/00_result.csv file...

compute 